# 036-KMP Speed up motif finding
## Shortening the motif search
- The standard approach to find whether one sequence appears as a substring of another is to slide the pattern along the text one position at a time and check for matches.
- If a mismatch occurs at the 100th position, we would need to go back 99 positions and start comparing again, which is inefficient.
- The computational complexity of this naive approach is approximately O(len(S) × len(T)).

## Example
- Let
    - t = ACGTACGT
    - s = TAGGTACGTACGGCATCACG

- In s[5:12], there are 7 matching characters.
- However, s[12] = G while t[7] = T, so a mismatch occurs.
- One might consider restarting the scan from s[6], but this is unnecessary.
- We already know that s[6:9] does not start with A, so it cannot match the prefix of t.
- If we start from s[9], we observe that
- t[0:4] = t[4:8] = ACGT,
- and since we already know s[12] ≠ t[7], a mismatch will inevitably occur at t[3].
- Therefore, s[9] should also be excluded as a candidate starting position, and the next valid starting point should be s[14].
- This idea forms the basis of the Knuth–Morris–Pratt (KMP) algorithm.

## Task
- Given a string s of length n, compute its Failure array P.

## Definitions
- For a string s:
  - A prefix is defined as s[1:j]
  - A suffix is defined as s[k:n]
  - (for simplicity, we consider 1-indexing)

- The failure array P[k] is defined as:
the length of the longest substring s[j:k] that is equal to some prefix s[1:k-j+1],
with the restriction j ≠ 1.

- The restriction j ≠ 1 is necessary because if j = 1, then s[j:k] = s[1:k], which would always match the prefix, resulting in P[k] = k for all k.

- Although this definition is somewhat difficult to understand
- It is equivalent to the partial match table used in the KMP algorithm.

## Alternative explanation
- Consider the substring s[1:k].
- Among all prefixes of this substring and suffixes of the same length,
the maximum length where they are equal is defined as P[k].
- By convention, P[1] = 0.

## Summary
- In essence, the task is to compute the partial match table (failure function) for a string of length n.

## Example
- For s = "ABABAB", the failure array is:

|index|pattern| Prefix + others | others + suffix | Failure array value(match_length) |
|:---:|:-----:|:---------------:|:---------------:|:-------------------:|
| 0 | A | - | - | 0 |
| 1 | AB | A + (b) | (a) + B | 0 |
| 2 | ABA | A + (ba) | (ab) + A | 1 |
| 3 | ABAB | AB + (ab) | (ab) AB | 2 |
| 4 | ABABA | ABA + (ab) | (ab) + ABA | 3 |
| 5 | ABABAB| ABAB + (ab) | (ab) + ABAB | 4 | 

- From this table, we can see that for indices [2–5],
instead of restarting the match from the beginning,
- if s[match_len + 1] = s[current_position], we can simply extend the existing match.

In [11]:
sample_seq = "CAGCATGGTATCACAGCAGAG"
worst_seq = "A" * 1000 + "B"

In [ ]:
import time
# at first, I try a naive implementation of failure table calculation
def naive_failure_table(seq):
    seq_len = len(seq)
    failure_array = [0] * seq_len

    for k in range(0, seq_len):
        longest_match_len = 0
        if k == 0 :
            failure_array[k] = 0
            continue
        for j in range(1, k + 1):
            # here, j is the length of the prefix/suffix to compare
            prefix = seq[0:j]
            suffix = seq[k - j + 1:k + 1]
            if prefix == suffix:
                longest_match_len = j
        failure_array[k] = longest_match_len
    return failure_array

# it can work correctly for sample sequence
failure_array_naive = naive_failure_table(sample_seq)
print("Naive failure array:", failure_array_naive)

# the purpose of KMP algorithm is to calculate failure table efficiently.
# but, this naive implementation is very slow, more than O(n^2).
time1 = time.time()
failure_array_naive = naive_failure_table(worst_seq)
time2 = time.time()

Naive failure array: [0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2, 3, 4, 5, 3, 0, 0]


In [ ]:
# From this table, we can see that for indices [2–5], instead of restarting the match from the beginning,
# if s[match_len + 1] = s[current_position], we can simply extend the existing match.

def efficient_failure_table(seq):
    seq_len = len(seq)
    failure_array = [0] * seq_len
    current_match_len = 0
    
    # Since P[0] = 0 by definition, we can start from index 1
    searching_index = 1
    while searching_index < seq_len:
        # If the next character matches, we can extend the current match
        # without backtracking.
        if seq[searching_index] == seq[current_match_len]:
            current_match_len += 1
            failure_array[searching_index] = current_match_len
            searching_index += 1
        # If the match cannot be extended, shorten the prefix length
        else:
            if current_match_len != 0:
                # Fall back to the next possible prefix length
                current_match_len = failure_array[current_match_len - 1]
            # If current_match_len is already 0, we cannot shorten further,
            # so we set failure_array[searching_index] to 0 and move on.
            else:
                failure_array[searching_index] = 0
                searching_index += 1
    return failure_array

# it can work correctly for sample sequence
failure_array_naive = naive_failure_table(sample_seq)
print("KMP failure array:", failure_array_naive)

time3 = time.time()
failure_array_efficient = efficient_failure_table(worst_seq)
time4 = time.time()


Naive failure array: [0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2, 3, 4, 5, 3, 0, 0]


In [16]:
# when we compare the time for both methods on the worst-case sequence,
# we can see that the efficient method is significantly faster.
print("Naive method time:", time2 - time1)
print("Efficient method time:", time4 - time3)

Naive method time: 0.0475153923034668
Efficient method time: 0.00012803077697753906
